In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the cleaned data file
file_path = "cleandata.csv"
df_clean = pd.read_csv(file_path)

# Convert the date column to datetime and set as index
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')

target = 'LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)'

df_forecast = df_clean[[target]].copy()
df_forecast.loc[df_forecast.index >= '2024-01-01', "Actual Rate"] = df_forecast[target].loc[df_forecast.index >= '2024-01-01']
df_forecast.loc[df_forecast.index >= '2024-01-01', "Naive: 1-Month"] = df_forecast[target].shift(1).loc[df_forecast.index >= '2024-01-01']
df_forecast.loc[(df_forecast.index >= '2024-01-01') & (df_forecast.index <= '2024-12-31'), "Naive: 12-Month"] = df_forecast[target][df_forecast.index <= "2023-12-01"].iloc[-1]

def add_naive_forecast(df, target_col, months_back, start_date, horizon_months, text):
    forecast_col = f"Naive: {months_back}-Month for {text}"
    df[forecast_col] = np.nan
    start = pd.to_datetime(start_date)
    end = start + pd.DateOffset(months=horizon_months - 1)
    reference_date = start - pd.DateOffset(months=months_back)
    if reference_date in df.index:
        naive_value = df[target_col][df.index < start_date].iloc[-1]
        forecast_range = (df.index >= start) & (df.index <= end)
        df.loc[forecast_range, forecast_col] = naive_value
    return df

# Adding 3-month naive forecasts
forecast_blocks_3m = [
    ('2024-01-01', 'Jan-Mar'), ('2024-02-01', 'Feb-Apr'), ('2024-03-01', 'Mar-May'),
    ('2024-04-01', 'Apr-Jun'), ('2024-05-01', 'May-Jul'), ('2024-06-01', 'Jun-Aug'),
    ('2024-07-01', 'Jul-Sep'), ('2024-08-01', 'Aug-Oct'), ('2024-09-01', 'Sep-Nov'),
    ('2024-10-01', 'Oct-Dec')
]
for start_date, text in forecast_blocks_3m:
    df_forecast = add_naive_forecast(df_forecast, target, months_back=3, start_date=start_date, horizon_months=3, text=text)

# Adding 6-month naive forecasts
forecast_blocks_6m = [
    ('2024-01-01', 'Jan-Jun'), ('2024-02-01', 'Feb-Jul'), ('2024-03-01', 'Mar-Aug'),
    ('2024-04-01', 'Apr-Sep'), ('2024-05-01', 'May-Oct'), ('2024-06-01', 'Jun-Nov'),
    ('2024-07-01', 'Jul-Dec')
]
for start_date, text in forecast_blocks_6m:
    df_forecast = add_naive_forecast(df_forecast, target, months_back=6, start_date=start_date, horizon_months=6, text=text)

# Adding 12-month naive forecasts
df_forecast = add_naive_forecast(df_forecast, target, months_back=12, start_date='2024-01-01', horizon_months=12, text="Jan-Dec")

# Calculating error metrics
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()
for forecast_type in df_forecast.columns[2:]:
    mask = ~df_forecast[forecast_type].isna()
    y_true = df_forecast.loc[mask, "Actual Rate"]
    y_pred = df_forecast.loc[mask, forecast_type]
    if not y_true.empty and not y_pred.empty:
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_type, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]),])
        # Adding individual errors for detailed view
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_type})
        error_details = pd.concat([error_details, details_df])

# Resetting index
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Extracting different forecast errors
naive_1_month_df = error_details[error_details["Forecast Type"] == "Naive: 1-Month"]
naive_3_month_df = error_details[error_details["Forecast Type"].str.contains("Naive: 3-Month")]
naive_6_month_df = error_details[error_details["Forecast Type"].str.contains("Naive: 6-Month")]
naive_12_month_df = error_details[error_details["Forecast Type"].str.contains("Naive: 12-Month")]

# Display the extracted dataframes
display(error_metrics)

print("1-Month Naive \n",
      "MAE :", round(mean_absolute_error(naive_1_month_df["Actual"],naive_1_month_df["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(naive_1_month_df['Actual'],naive_1_month_df["Forecast"])),2))
print("3-Month Naive \n",
      "MAE :", round(mean_absolute_error(naive_3_month_df["Actual"],naive_3_month_df["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(naive_3_month_df['Actual'],naive_3_month_df["Forecast"])),2))
print("6-Month Naive \n",
      "MAE :", round(mean_absolute_error(naive_6_month_df["Actual"],naive_6_month_df["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(naive_6_month_df['Actual'],naive_6_month_df["Forecast"])),2))
print("12-Month Naive \n",
      "MAE :", round(mean_absolute_error(naive_12_month_df["Actual"],naive_12_month_df["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(naive_12_month_df['Actual'],naive_12_month_df["Forecast"])),2))


/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_17071/2323512275.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_type, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]),])


,Forecast Type,MAE,RMSE
0,Naive: 1-Month,14864.583333,23265.119972
1,Naive: 12-Month,85422.916667,87031.788374
2,Naive: 3-Month for Jan-Mar,81058.333333,81437.154962
3,Naive: 3-Month for Feb-Apr,18454.166667,18653.112173
4,Naive: 3-Month for Mar-May,2687.500000,3079.307620
5,Naive: 3-Month for Apr-Jun,6333.333333,6807.326017
6,Naive: 3-Month for May-Jul,15629.166667,18636.043985
7,Naive: 3-Month for Jun-Aug,21191.666667,22048.651225
8,Naive: 3-Month for Jul-Sep,9808.333333,10732.704109
9,Naive: 3-Month for Aug-Oct,13350.000000,18165.079250


1-Month Naive 
 MAE : 14864.58 
 RMSE: 23265.12
3-Month Naive 
 MAE : 23437.08 
 RMSE: 32959.16
6-Month Naive 
 MAE : 25291.96 
 RMSE: 36017.86
12-Month Naive 
 MAE : 85422.92 
 RMSE: 87031.79
